In [1]:
!pip install torchinfo

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [19]:
import os
#set workdir
os.chdir("/content/gdrive/My Drive/Projects/MSc-Project-River-Segmentation/MSc-Project/Code")

In [5]:
! ls


config.cfg  dataloader.py  models  __pycache__	Testing.ipynb  Training.ipynb


In [6]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from dataloader import Dataset
from torchinfo import summary
import time
import copy
import pdb
from tqdm import tqdm

In [7]:
#training parameters in neptune format
PARAMS = {
    "input_size": 416,
    "output_size": 416,
    "model": "vgg_unet",
    "learning_rate": 0.0001,
    "batch_size": 8,
    'epochs': 1000,
    'patience': 10,
    "train_dataset_size": -1, # set train dataset subset. Useful when neet to 
                              # overtrain model with small amount of images.
                              # -1 -all images from train directories.
    "test_dataset_size": -1,  # set test dataset subset.
                              # -1 -all images from train directories.
    "n_classes": 2,
    'image_preload': False,
}

In [8]:
import configparser
config = configparser.ConfigParser()
config.read("./config.cfg")

['./config.cfg']

In [9]:
#neptune installation and initialization
!pip install neptune-client
import neptune
neptune.init(
             project_qualified_name='moad.s.k/MSc-Project',
             api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI1MWE3YmU1ZC1jNWViLTQ1ZjUtYTE4NC1mOTgzYmUxNmI5MDkifQ==' # your credentials

             #project_qualified_name=config["neptune"]["project"],
             #api_token=config["neptune"]["token"],
             )
neptune.create_experiment(params=PARAMS)

     |████████████████████████████████| 252 kB 7.0 MB/s 
     |████████████████████████████████| 829 kB 13.6 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 170 kB 28.3 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 67 kB 7.0 MB/s 
     |████████████████████████████████| 129 kB 28.1 MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.10.4-py2.py3-none-any.whl size=432189 sha256=04c3174a34d5542588748526b5dd5044a5d6601bceda256a9b31ff45d9b237ce
  Stored in directory: /root/.cache/pip/wheels/b0/9c/47/8f8a6f14aa49ca017a36ad440e83c31c9539e93a14aae138da
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=1b18bb481f4ce3c2bcbd3bef42fd7f87e9a1773ded1d825d570f6d0c1c6574b4
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
  Created wheel for strict-rfc3339: filename=strict_rfc3339-0.7

Experiment(MSCPROJ-6)

In [17]:
os.chdir("/content/gdrive/My Drive/Data/sentinel-river-segmentation-dataset/sentinel-river-segmentation-dataset")

In [18]:
! ls


README.md  x_test  x_train  y_test  y_train


In [20]:
#dataset configuration

import os

dataset_dir = os.path.normpath("/content/gdrive/My Drive/Data/sentinel-river-segmentation-dataset/sentinel-river-segmentation-dataset")
x_train_dir = os.path.join(dataset_dir,"x_train")
y_train_dir = os.path.join(dataset_dir,"y_train")
x_test_dir = os.path.join(dataset_dir,"x_test")
y_test_dir = os.path.join(dataset_dir,"y_test")

train_set = Dataset(x_train_dir, y_train_dir, input_size=PARAMS['input_size'], output_size=PARAMS['output_size'], n_classes=PARAMS["n_classes"], count=PARAMS["train_dataset_size"])
test_set = Dataset(x_test_dir, y_test_dir, input_size=PARAMS['input_size'], output_size=PARAMS['output_size'], n_classes=PARAMS["n_classes"], count=PARAMS["test_dataset_size"])

batch_size = PARAMS['batch_size']
dataloaders = {
    'train': DataLoader(train_set, batch_size=PARAMS['batch_size'], shuffle=True, num_workers=0),
    'val': DataLoader(test_set, batch_size=PARAMS['batch_size'], shuffle=True, num_workers=0)
}

In [21]:
# load images - useful if you want to save some time by preloading images (very time-consuming) when 
# the model is still not fuctional and cant run standard training.
if PARAMS['image_preload']:
    for phase in dataloaders:
        for inputs, labels in tqdm(dataloaders[phase]):
            pass

In [22]:
#model loading
if PARAMS['model'] == "simple":
    from models.simple import Simple
    model = Simple()
elif PARAMS['model'] == "vgg_unet":
    from models.vgg_unet import VggUnet
    model = VggUnet()
elif PARAMS['model'] == "vgg_unet_ks":
    from models.vgg_unet_ks import VggUnetKs
    model = VggUnetKs()
elif PARAMS['model'] == "unet":
    from models.unet import UNet
    model = UNet()
elif PARAMS['model'] == "vgg_deconvnet":
    from models.vgg16_deconvnet import VggDeconvNet
    model = VggDeconvNet()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


In [23]:
#model structure preview

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model_stats = summary(model, input_size=(PARAMS['batch_size'], 3, PARAMS['input_size'], PARAMS['input_size']))
for line in str(model_stats).splitlines():
    neptune.log_text('model_summary', line)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [24]:
from collections import defaultdict
import torch.nn.functional as F
SMOOTH = 1e-6
def iou_metric(outputs: torch.Tensor, labels: torch.Tensor):
    outputs = outputs[:,1,:,:]  # BATCH x 1 x H x W => BATCH x H x W
    labels = labels[:,1,:,:]
    intersection = (outputs * labels).sum(2).sum(1)  # Will be zero if Truth=0 or Prediction=0
    union = (outputs + labels).sum(2).sum(1) - intersection  # Will be zzero if both are 0
    iou = (intersection + SMOOTH) / (union + SMOOTH)  # We smooth our devision to avoid 0/0
    return iou.mean()
    

def dice_loss(pred, target, smooth = 1.):
    pred = pred.contiguous()
    target = target.contiguous()    
    intersection = (pred * target).sum(dim=2).sum(dim=2)
    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))
    return loss.mean()

def calc_loss(pred, target, metrics, bce_weight=0.5):
    bce = F.binary_cross_entropy(pred, target)
    pred = torch.round(pred)
    dice = dice_loss(pred, target)
    loss = bce# * bce_weight + dice * (1 - bce_weight)
    iou = iou_metric(pred, target)
    metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
    metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)
    metrics['iou'] += iou.data.cpu().numpy() * target.size(0)
    return loss

def print_metrics(metrics, epoch_samples, phase):   
    print(epoch_samples) 
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))
        neptune.log_metric(phase+"_"+k, metrics[k] / epoch_samples) #log
    print("{}: {}".format(phase, ", ".join(outputs)))    



In [25]:
#training loop
def train_model(model, dataloaders, optimizer, device, num_epochs=25, patience=-1):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_accuracy = 0
    no_improvement = 0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        since = time.time()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])
                    
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0
            
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)             

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = calc_loss(outputs, labels, metrics)
                    #print(model.encoder[0].weight.grad)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        #pdb.set_trace()
                        optimizer.step()

                # statistics
                epoch_samples += inputs.size(0)

            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['loss'] / epoch_samples
            epoch_accuracy = metrics['iou'] / epoch_samples

            # deep copy the model
            if phase == 'val':
                if epoch_accuracy > best_accuracy:
                    no_improvement = 0
                    print("Val IoU improved by {}. Saving best model.".format(epoch_accuracy-best_accuracy))
                    best_accuracy = epoch_accuracy
                    best_model_wts = copy.deepcopy(model.state_dict())
                else:
                    no_improvement += 1
                    print("No accuracy improvement since {}/{} epochs.".format(no_improvement,patience))
        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        if patience >= 0 and no_improvement > patience:
            break
    print('Best accuracy: {:4f}'.format(best_accuracy))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
#model training
optimizer_ft = optim.Adam(model.parameters(), lr=PARAMS['learning_rate'])
model = train_model(model, dataloaders, optimizer_ft, device, num_epochs=PARAMS['epochs'], patience=PARAMS['patience'])

In [44]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Training.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [31]:
!git add .

In [32]:
!git commit -m "updated"
!git config --global user.email "moad.s.k@gmail.com"
!git config --global user.name "MoadSaadKhorchef"

[main 5e09488] updated
 3 files changed, 1 insertion(+), 365 deletions(-)
 rewrite Code/Training.ipynb (98%)
 create mode 100644 Code/__pycache__/dataloader.cpython-37.pyc
 create mode 100644 Code/models/__pycache__/vgg_unet.cpython-37.pyc


In [35]:
!git push origin main

Counting objects: 9, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (9/9), done.
Writing objects: 100% (9/9), 10.83 KiB | 2.17 MiB/s, done.
Total 9 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/MoadSaadKhorchef/MSc-Project.git
   0a4fec6..5e09488  main -> main
